# Load Sentinel-2 data from the CDSE STAC catalog

This notebook shows an example how to load Sentinel-2 data from the [CDSE STAC API](https://browser.stac.dataspace.copernicus.eu/?.language=en).

### Setup
In order to run this notebook you may install [`xcube_stac`](https://github.com/xcube-dev/xcube-stac), following the [README](../../README.md).

The data can be accessed via S3, where key and secret can be obtained following the [CDSE access documentation to EO data via S3](https://documentation.dataspace.copernicus.eu/APIs/S3.html#generate-secrets). The store object will receive the key and secret upon initialization, as demonstrated below.

Now, we first import everything we need:

In [1]:
%%time
import itertools

import matplotlib.pyplot as plt
from xcube.core.store import new_data_store, get_data_store_params_schema

from xcube_stac.utils import reproject_bbox

CPU times: user 3.46 s, sys: 263 ms, total: 3.72 s
Wall time: 1.83 s


Next store the credentials in a dictionary. 

In [2]:
credentials = {
    "key": "O0M0CUQIDQO9TDZ4D8NR",
    "secret": "qPUyXs9G6j8on6MY5KPhQNHuA5uZTqxEscrbBCGx",
}

First, we get the store parameters needed to initialize a STAC [data store](https://xcube.readthedocs.io/en/latest/dataaccess.html#data-store-framework). Note that key and secret of the S3 access are required.

In [3]:
%%time
store_params = get_data_store_params_schema("stac-cdse")
store_params

CPU times: user 18.7 ms, sys: 11 ms, total: 29.7 ms
Wall time: 29.4 ms


Note that the user does not need to provide the URL for the [CDSE STAC API](https://documentation.dataspace.copernicus.eu/APIs/STAC.html). Only the key and secret for S3 access are required when initializing a `stac-cdse` [data store](https://xcube.readthedocs.io/en/latest/dataaccess.html#data-store-framework). First, we will initialize a store supporting the stacking mode. Then, for completeness, we will initialize a store in single-tile mode.

In [4]:
%%time
store = new_data_store("stac-cdse", stack_mode=True, **credentials)

CPU times: user 9.63 ms, sys: 0 ns, total: 9.63 ms
Wall time: 171 ms


The data IDs point to a STAC collections. So far only `'sentinel-2-l2a'` is supported.

In [5]:
%%time
data_ids = store.list_data_ids()
data_ids

CPU times: user 60.8 ms, sys: 1.07 ms, total: 61.9 ms
Wall time: 393 ms


['sentinel-3-olci-2-wfr-nrt',
 'sentinel-1-mosaic',
 'sentinel-3-sl-2-aod-nrt',
 'sentinel-1-grd',
 'sentinel-2-l2a',
 'sentinel-2-l1c',
 'sentinel-2-global-mosaics',
 'sentinel-3-sl-2-wst-nrt',
 'sentinel-3-olci-1-efr-nrt',
 'sentinel-3-olci-2-wrr-nrt',
 'sentinel-3-sl-2-lst-ntc',
 'sentinel-3-olci-1-efr-ntc',
 'sentinel-3-olci-2-wfr-ntc',
 'sentinel-3-olci-2-lfr-nrt',
 'sentinel-3-sl-2-wst-ntc',
 'sentinel-3-olci-1-err-nrt',
 'sentinel-3-olci-2-lfr-ntc',
 'sentinel-3-sl-2-frp-ntc',
 'sentinel-3-olci-1-err-ntc',
 'sentinel-3-sl-1-rbt-ntc',
 'sentinel-3-olci-2-lrr-ntc',
 'sentinel-3-sl-2-lst-nrt',
 'sentinel-3-sl-1-rbt-nrt',
 'sentinel-3-olci-2-lrr-nrt',
 'sentinel-3-sl-2-frp-nrt',
 'sentinel-5p-l1-ra-bd2-offl',
 'sentinel-3-sr-1-sra-a-nrt',
 'sentinel-5p-l1-ra-bd2-nrti',
 'sentinel-3-sr-1-sra-a-stc',
 'sentinel-5p-l1-ra-bd1-rpro',
 'sentinel-3-sr-1-sra-a-ntc',
 'sentinel-3-sr-1-sra-bs-ntc',
 'sentinel-5p-l1-ra-bd5-nrti',
 'sentinel-3-sr-1-sra-bs-stc',
 'sentinel-5p-l1-ra-bd6-offl',
 '

Below, the parameters for the `open_data` method can be viewed.

In [6]:
%%time
open_params = store.get_open_data_params_schema()
open_params

CPU times: user 29 μs, sys: 3 μs, total: 32 μs
Wall time: 33.4 μs


So far, only data from the collection `sentinel-2-l2a` can be accessed. We therefore assign `data_id` to `"sentinel-2-l2a"`. We set the bounding box to cover the greater Hamburg area and the time range to second half of July 2020.

In [ ]:
%%time
ds = store.open_data(
    data_id="sentinel-2-l2a",
    bbox=[9.1, 53.1, 10.7, 54],
    time_range=["2020-07-15", "2020-08-01"],
    spatial_res=10 / 111320, # meter in degree
    crs="EPSG:4326",
    asset_names=["B02", "B03", "B04", "SCL"],
    apply_scaling=True,
    angles_sentinel2=True,
)
ds

We can plot the B04 (red) band for a given timestamp as an example. Hereby a mosaicking of multiple tiles have been applied. Additionally, we plot the solar and viewing angle.


In [ ]:
%%time
fig, ax = plt.subplots(1, 3, figsize=(20, 6))
ds.B04.isel(time=-1)[::10, ::10].plot(ax=ax[0], vmin=0, vmax=0.2)
ds.solar_angle.isel(angle=0, time=-1).plot(ax=ax[1])
ds.viewing_angle.isel(band=2, angle=0, time=-1).plot(ax=ax[2])

The data access can be speed up when requesting the data in the UTM CRS which is the native UTM of the Sentinel-2 products. 

In [8]:
%%time
bbox = [9.1, 53.1, 10.7, 54]
crs_target = "EPSG:32632"
bbox_utm = reproject_bbox(bbox, "EPSG:4326", crs_target)

CPU times: user 1.1 ms, sys: 6 μs, total: 1.11 ms
Wall time: 558 μs


In [9]:
%%time
ds = store.open_data(
    data_id="sentinel-2-l2a",
    bbox=bbox_utm,
    time_range=["2020-07-15", "2020-08-01"],
    spatial_res=10,
    crs=crs_target,
    asset_names=["B02", "B03", "B04", "SCL"],
    apply_scaling=True,
    angles_sentinel2=True,
)
ds

CPU times: user 32.6 s, sys: 1.21 s, total: 33.8 s
Wall time: 1min 10s


<xarray.Dataset> Size: 17GB
Dimensions:        (time: 11, y: 10147, x: 10727, angle_x: 23, angle_y: 22,
                    angle: 2, band: 3)
Coordinates:
  * time           (time) datetime64[ns] 88B 2020-07-15T10:15:59.024000 ... 2...
    spatial_ref    int64 8B 0
  * x              (x) float64 86kB 5.066e+05 5.066e+05 ... 6.138e+05 6.138e+05
  * y              (y) float64 81kB 5.985e+06 5.985e+06 ... 5.883e+06 5.883e+06
  * angle_x        (angle_x) float64 184B 5.066e+05 5.116e+05 ... 6.166e+05
  * angle_y        (angle_y) float64 176B 5.988e+06 5.983e+06 ... 5.883e+06
  * angle          (angle) object 16B 'zenith' 'azimuth'
  * band           (band) <U3 36B 'B02' 'B03' 'B04'
Data variables:
    B02            (time, y, x) float32 5GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B03            (time, y, x) float32 5GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B04            (time, y, x) float32 5GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    SCL            (time, y, x) uint16 2GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    solar_angle    (angle, time, angle_y, angle_x) float32 45kB dask.array<chunksize=(2, 1, 22, 23), meta=np.ndarray>
    viewing_angle  (angle, band, time, angle_y, angle_x) float32 134kB dask.array<chunksize=(2, 3, 1, 22, 23), meta=np.ndarray>
Attributes:
    stac_item_ids:     {'2020-07-15T10:15:59.024000': ['S2B_MSIL2A_20200715T1...
    stac_catalog_url:  https://stac.dataspace.copernicus.eu/v1

Note that the search function in the CDSE STAC API is very slow. Further investigation and comparison with other STAC APIs is needed. 

We can plot the B04 (red) band for a given timestamp as an example. Hereby a mosaicking of multiple tiles have been applied. Additionally, we plot the solar and viewing angle.

In [ ]:
%%time
fig, ax = plt.subplots(1, 3, figsize=(20, 6))
ds.B04.isel(time=-1)[::10, ::10].plot(ax=ax[0], vmin=0, vmax=0.2)
ds.solar_angle.isel(angle=0, time=-1).plot(ax=ax[1])
ds.viewing_angle.isel(band=2, angle=0, time=-1).plot(ax=ax[2])

----
## Data store in the single-tile mode
For completeness, we initiate the data store in the single-tile mode and open data of one tile. 

In [ ]:
%%time
store = new_data_store("stac-cdse", stack_mode=False, **credentials)

The data IDs point to a [STAC item's JSON](https://github.com/radiantearth/stac-spec/blob/master/item-spec/item-spec.md) and are specified by the segment of the URL that follows the catalog's URL. The data IDs can be streamed using the following code where we show the first 10 data IDs as an example.

⚠️ Warning: If you use `store.list_data_ids()` it will try to collect all Sentinel-2 tiles in the archive, before printing the result. This can take a while, and is not recommended. 

In [ ]:
%%time
data_ids = store.get_data_ids()
list(itertools.islice(data_ids, 10))

In the next step, we can search for items using search parameters. The following code shows which search parameters are available.

In [ ]:
%%time
search_params = store.get_search_params_schema()
search_params

 Next, we will search for tiles of Sentinel-2 data.

In [ ]:
%%time
descriptors = list(
    store.search_data(
        collections=["sentinel-2-l2a"],
        bbox=[9, 47, 10, 48],
        time_range=["2020-07-01", "2020-07-05"],
    )
)
[d.to_dict() for d in descriptors]

In the next step, we can open the data for each data ID. The following code shows which parameters are available for opening the data.

In [ ]:
%%time
open_params = store.get_open_data_params_schema()
open_params

We select the band B04 (red), B03 (green), B02 (blue), and the science classification layer (SLC), and lazily load the corresponding data.

In [ ]:
%%time
ds = store.open_data(
    "collections/sentinel-2-l2a/items/S2B_MSIL2A_20200705T101559_N0500_R065_T32TMT_20230530T175912",
    asset_names=["B04", "B03", "B02", "SCL"],
    apply_scaling=True,
    angles_sentinel2=True,
)
ds

We plot the loaded data as an example below.

In [ ]:
%%time
ds.B04[::10, ::10].plot(vmin=0.0, vmax=0.2)

We can also open a `.jp2` as a [xcube's multi-resolution  dataset](https://xcube.readthedocs.io/en/latest/mldatasets.html#xcube-multi-resolution-datasets), where we can select the level of resolution, shown below.  

In [ ]:
%%time
mlds = store.open_data(
    descriptors[3].data_id,
    data_type="mldataset",
    asset_names=["B04", "B03", "B02"],
    apply_scaling=True,
    angles_sentinel2=True,
)
mlds.num_levels

In [ ]:
%%time
ds = mlds.get_dataset(2)
ds

In [ ]:
%%time
ds.B04[::10, ::10].plot(vmin=0.0, vmax=0.2)